In [1]:
# Importação das bibliotecas
import pyvis as pv
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import random as rd
import numpy as np
from matplotlib.pyplot import figure

In [2]:
# Importando as áreas
raw_data = pd.read_csv("c:/Users/Usuario/Documents/Estudos/Data Science e Programação/Datasets/colaboradores.csv")
raw_data = raw_data[['nome', 'area']]

In [3]:
# Criando uma lista de quantidade de pessoas por área
areas = set(raw_data['area'])

In [4]:
# Criando uma função para encontrar a estimativa do número de conecções 
def total_conections(name):
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    total = len(raw_data.loc[raw_data['area'] == area])
    conections = rd.randint(round(total*0.1), round(total*1.5))
    return conections

In [5]:
# Criando uma função para atribuir a ponderação de cada nome
def weight_definition(name):
    # Encontrando a área
    areas_dict = dict.fromkeys(areas, 1)
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    # Atribuindo o peso adequado
    areas_dict[area] = 50
    raw_data['probabilidade'] = raw_data.apply(lambda x: areas_dict[x['area']], axis = 1)
    # Criando o vetor de itens e pesos
    raw_data_filter = raw_data.loc[raw_data['nome'] != name]
    itens = list(raw_data_filter['nome'])
    probability = list(raw_data_filter['probabilidade'])
    weights = [j/sum(probability) for j in probability]
    return itens, weights

In [6]:
# Criando uma função para criar a lista de conecções de cada pessoa
def ona_conections(name):
    # Total de conecções
    conections = total_conections(name)
    # Definição da lista de nomes e pesos
    names_list = weight_definition(name)[0]
    weights_list = weight_definition(name)[1]
    # Criação da lista de coneções
    conections_list = np.random.choice(a = names_list, size = conections, p = weights_list)
    return conections_list  

In [7]:
# Criando uma função para gerar a rede de cada nome
def ona_generator(name):
    conections_list = ona_conections(name)
    ona_list = [(name, list(raw_data.loc[raw_data['nome'] == name]['area'])[0], i, list(raw_data.loc[raw_data['nome'] == i]['area'])[0]) for i in conections_list]
    return ona_list

In [8]:
# Construindo a rede organizacional
ona_total_list = []
for i in list(raw_data['nome']):
    ona_total_list.extend(ona_generator(i))

In [9]:
# Criação do dataset
dataset = pd.DataFrame(ona_total_list, columns = ['partida', 'area_partida', 'chegada', 'area_chegada'])

In [10]:
# Criação dos vetores principais
source = list(dataset['partida'])
source_area = list(dataset['area_partida'])
target = list(dataset['chegada'])
target_area = list(dataset['area_chegada'])

In [11]:
# Criação dos vetores de atributos
source_size = [source.count(i) for i in source]
target_size = [target.count(i) for i in target]

In [12]:
probability_edge_same_area = [0.1, 0.1, 0.2, 0.4, 0.2]
probability_edge_different_area = [0.4, 0.2, 0.2, 0.1, 0.1]
edge_weights_list = [1, 2, 3, 4, 5]
edge_weights = []
for i in range(len(source)):
    if source_area[i] == target_area[i]:
        edge_weights.append(np.random.choice(a = edge_weights_list, size = 1, p = probability_edge_same_area)[0])
    else:
        edge_weights.append(np.random.choice(a = edge_weights_list, size = 1, p = probability_edge_different_area)[0])

In [13]:
# Criação da lista descritiva de cada conexão
ona_edge = list(zip(source, source_area, source_size, target, target_area, target_size, edge_weights))

In [29]:
graph = Network(height='1200px', width='100%', bgcolor='#111111', font_color='white', directed = True)
graph.barnes_hut(gravity = -80000, central_gravity=0.3, spring_length = 250, spring_strength=0.001, damping=0.09, overlap = 0)

In [30]:
for i in ona_edge:
    source = i[0]
    source_area = i[1]
    source_size = int(i[2])
    target = i[3]
    target_area = i[4]
    target_size = int(i[5])
    weight = int(i[6])
    
    if souce_size > 25:
        source_mass = 10
    else:
        source_mass = 3

    if target_size > 25:
        target_mass = 10
    else:
        target_mass = 3
    
    graph.add_node(source, title = source, group = source_area, size = source_size, mass = source_mass)
    graph.add_node(target, title = target, group = target_area, size = target_size, mass = target_mass)
    graph.add_edge(source, target, width = weight, color = "#202020")

In [28]:
graph.show('graph.html')